In [1]:
import torch
from accelerate import Accelerator, DistributedDataParallelKwargs
import transformers
from tqdm import tqdm
import os

In [2]:
from huggingface_hub import notebook_login

# Handle lighting AI studio
if '/teamspace' in os.getcwd():
    os.chdir('/teamspace/studios/this_studio/llm2vec-da')
    print(os.getcwd())

notebook_login()

/teamspace/studios/this_studio/llm2vec-da


In [ ]:
from llm2vec_da.arguments import simcse_parser

    
model_args, data_args, training_args, custom_args = simcse_parser.parse_json_file(
        "llama-swe-it-simcse.json"
    )
accelerator = Accelerator()
transformers.set_seed(training_args.seed)
if training_args.gradient_checkpointing:
    training_args.gradient_checkpointing_kwargs = {"use_reentrant": False}

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ValueError: Your setup doesn't support bf16/gpu. You need torch>=1.10, using Ampere GPU with cuda>=11.0

### Load the model

In [3]:
from llm2vec_da.data_utils import PairedDataset, load_raw_datasets

In [4]:
datasets = load_raw_datasets(data_args, model_args)
train_dataset = PairedDataset(datasets['train'])
valid_dataset = PairedDataset(datasets['validation'])
train_examples = [train_dataset[i] 
                  for i in tqdm(range(len(train_dataset)),desc="Loading train examples...",disable=not accelerator.is_main_process)
]
validation_examples = [
    valid_dataset[i]
    for i in tqdm(
        range(len(valid_dataset)),
        desc="Loading train examples...",
        disable=not accelerator.is_main_process,
    )
]

NameError: name 'data_args' is not defined

### Loading the model

In [ ]:
from llm2vec_da import LLM2Vec

model = LLM2Vec.from_pretrained(
    base_model_name_or_path=model_args.model_name_or_path,
    enable_bidirectional=model_args.bidirectional,
    peft_model_name_or_path=model_args.peft_model_name_or_path,
    merge_peft=True,
    pooling_mode=model_args.pooling_mode,
    max_length=model_args.max_seq_length,
    torch_dtype=getattr(torch, model_args.torch_dtype),
    attn_implementation=model_args.attn_implementation,
    attention_dropout=custom_args.simcse_dropout,
    load_in_8bit=True
)
model

ValueError: FlashAttention2 has been toggled on, but it cannot be used due to the following error: Flash Attention 2 is not available on CPU. Please make sure torch can access a CUDA device.

## Training

In [ ]:
from llm2vec.loss.utils import load_loss
train_loss = load_loss(custom_args.loss_class, scale=custom_args.loss_scale)
train_loss

In [ ]:
from bg2vec.training import SimCSEDefaultCollator
data_collator = SimCSEDefaultCollator(model.tokenize)

In [ ]:
from bg2vec.training import SimCSETrainer, StopTrainingCallback
trainer = SimCSETrainer(
    model=model,
    args=training_args,
    train_dataset=train_examples,
    eval_dataset=validation_examples,
    data_collator=data_collator,
    tokenizer=model.tokenizer,
    loss_function=train_loss,
)

if custom_args.stop_after_n_steps is not None:
    trainer.add_callback(StopTrainingCallback(custom_args.stop_after_n_steps))
trainer.callback_handler.remove_callback(transformers.integrations.integration_utils.WandbCallback)

In [ ]:
trainer.train()